In [1]:
#test get_sequence
import os, pickle
import numpy as np
import Bio.PDB
from Bio import SeqIO
from Bio.PDB.DSSP import DSSP
from collections import defaultdict
import numpy as np
import torch
from torch.utils.data import Dataset
from Utils import struc_feat
#import from atomRefine cal_covalent and bond and 
from Utils.pair_handler import cal_covalent
import os,re

In [24]:
class DataConversion(Dataset):
    def __init__(self, root, transform=None, pre_transform=None):
        # super(DataConversion, self).__init__(root, transform=None,
        #         pre_transform=None)
        #TO DO: possible pre-relax transformation protocol?
        self.root = root
        
    def __len__(self):
        return self.data_len
    
    @property
    def raw_pre_relaxed_files(self):
        return [filename for filename in os.scandir(self.root + "/pre_relax") ]
    @property
    def raw_relaxed_afdb_files(self):
        return [filename for filename in os.scandir(self.root + "/relax") if "_relaxed_0001.pdb" in filename.name]
    
    
    def __get_node_feat__(self, file):
        #TODO call get_backbone_angles for pre-relax and relax
        # node feat
        node_feat = {}
        #for file in self.raw_pre_relaxed_files[-4:]:
        pre_relax_file  = file
        id = os.path.splitext(os.path.basename(file))[0]
        relax_file = self.root + "/relax/" + id + "_relaxed_0001.pdb"
        print(id, relax_file)
        # id_relax = id = os.path.splitext(os.path.basename(file))[0].replace("_relaxed_0001", "")
        # atom_emb
        #get residue sequence and length of amino acid sequence and feed it into the embedding extractor 
        residue_sequence, seq_len = struc_feat.get_sequence(pre_relax_file)
        atom_embs, atom_xyz, atom_relax_xyz, atom_nums, ca_pose= struc_feat.get_atom_emb(pre_relax_file, relax_file, [1, seq_len], residue_sequence)
        
        #get residue embeddings to concatenate iwth atom emb
        residue_onehot = struc_feat.residue_onehot(residue_sequence)
        #add residue encoding to atom emb
        for i in range(len(atom_embs)):
            #assign a seq emb to every atom of that residue
            seq_emb = np.tile(residue_onehot[i], (len(atom_embs[i]), 1))
            atom_emb = atom_embs[i]
            atom_embs[i] = np.concatenate((seq_emb, atom_emb[:, :]), axis=1)
            
        #get per-residue atom coordinates
        
        node_feat['atom_emb'] = {
            'embedding': atom_embs, #one_hot_atom_id +  one_hot_residue_id
            'atom_pos': atom_xyz,
            'atom_relax_pos': atom_relax_xyz,
            'atom_nums': atom_nums,
            'residue_sequence': residue_sequence,
            'CA_lst': ca_pose,
        }
        return node_feat
    
    """add this new edge handler function"""
    def __get_edge_feat(self, file):
        #TODO: call 
        return 

In [27]:
for item in DataConversion(root = "/work/slusky/s300y051/relax/test").raw_pre_relaxed_files[-4:]:
    node_feat = DataConversion(root = "/work/slusky/s300y051/relax/test").__get_node_feat__(item)

A0A009IHW8 /work/slusky/s300y051/relax/test/relax/A0A009IHW8_relaxed_0001.pdb
A0A023I7E1 /work/slusky/s300y051/relax/test/relax/A0A023I7E1_relaxed_0001.pdb
A0A023YYV9 /work/slusky/s300y051/relax/test/relax/A0A023YYV9_relaxed_0001.pdb
A0A059TC02 /work/slusky/s300y051/relax/test/relax/A0A059TC02_relaxed_0001.pdb


In [ ]:
#make pdb parser
pdb_parser = Bio.PDB.PDBParser(QUIET=True)
structure_true = pdb_parser.get_structure('native', "/work/slusky/s300y051/relax/test/pre_relax/A0A009IHW8.pdb")
for res in structure_true.get_residues():
    _resname = res.get_resname()
    print(res.id[1])
    
    # for _atom in heavy_atoms_per_residue[_resname]['atoms']:
    #     #get coord
    #     atom_coord = res[_atom].coord
    #     print(atom_coord)
      
        


        
    

Get atomic neighborhood emb/distance based edges

Calculate Covalent edges